In [1]:
start = 1000000
stop = 2000000

In [ ]:
!pip install pandas

In [1]:
import numpy as np
import pandas as pd
import h5py
import os

In [5]:
!wget https://zenodo.org/record/6544649/files/train_sequences.txt?download=1
!mv train_sequences.txt?download=1 train_sequences.txt

--2022-06-08 12:27:31--  https://zenodo.org/record/6544649/files/train_sequences.txt?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 805369640 (768M) [text/plain]
Saving to: ‘train_sequences.txt?download=1’

train_sequences.txt 100%[===================>] 768.06M  4.57MB/s    in 2m 8s   

2022-06-08 12:29:41 (5.99 MB/s) - ‘train_sequences.txt?download=1’ saved [805369640/805369640]



In [1]:
!wget https://zenodo.org/record/4436477/files/complex_media_training_data_Glu.txt?download=1
!mv complex_media_training_data_Glu.txt?download=1 data.text

--2022-06-09 19:58:13--  https://zenodo.org/record/4436477/files/complex_media_training_data_Glu.txt?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3592193655 (3.3G) [text/plain]
Saving to: ‘complex_media_training_data_Glu.txt?download=1’

complex_media_train 100%[===================>]   3.34G  8.88MB/s    in 3m 39s  

2022-06-09 20:01:53 (15.6 MB/s) - ‘complex_media_training_data_Glu.txt?download=1’ saved [3592193655/3592193655]



In [2]:
with open('train_sequences.txt') as handle:
    raw_data = pd.read_csv(handle, lineterminator='\n').to_numpy()

In [3]:
max_length = 0

def split_inputs_outputs(a):
    global max_length
    element = a.split('\t')
    seq, label = element[0], element[1]
    pre = 'TGCATTTTTTTCACATC'
    post = 'GGTTACGGCTGTT'
    seq = seq[len(pre):-len(post)]
    max_length = len(seq) if len(seq) > max_length else max_length
    return seq, label

func = np.vectorize(split_inputs_outputs)
temp = np.squeeze(raw_data)
inputs, outputs = func(temp)
outputs = outputs.astype(np.dtype('f8'))

: 

: 

In [ ]:
def find_Ns(s):
    if 'N' in s:
        return 0
    else:
        return 1

func = np.vectorize(find_Ns)
mask = func(inputs).astype(bool)
spliced_inputs = inputs[mask]
spliced_outputs = outputs[mask]
print(spliced_inputs.shape)

In [ ]:
def one_hot_encode(seq, L, i):
    if i % 100000 == 0:
        print(i//100000)
    seq = np.array(list(seq))
    encoded = np.zeros((L, 4))
    As = np.where(seq == 'A')[0]
    Cs = np.where(seq == 'C')[0]
    Ts = np.where(seq == 'T')[0]
    Gs = np.where(seq == 'G')[0]
    encoded[As] = np.array([1, 0, 0, 0])
    encoded[Cs] = np.array([0, 1, 0, 0])
    encoded[Gs] = np.array([0, 0, 1, 0])
    encoded[Ts] = np.array([0, 0, 0, 1])
    return encoded

order = np.arange(spliced_inputs.shape[0])
one_hot_inputs = np.vectorize(one_hot_encode, otypes=[np.ndarray])(spliced_inputs[start:stop], max_length, order[start:stop])
one_hot_inputs = np.array(one_hot_inputs.tolist(), dtype=np.dtype('f8'))
print(one_hot_inputs.shape)

In [ ]:
print(spliced_outputs[1])

In [ ]:
if os.path.exists('dataset.h5'):
    os.remove('dataset.h5')
    
hf = h5py.File('dataset.h5', 'w')
print('created file')
hf.create_dataset('x_train', data=one_hot_inputs[0:700000], maxshape=(spliced_outputs.shape[0], max_length, 4), chunks=True)
print('saved training inputs')
hf.create_dataset('y_train', data=spliced_outputs[1000000:1700000], maxshape=(spliced_outputs.shape[0],), chunks=True)
print('saved training outputs')
hf.create_dataset('x_valid', data=one_hot_inputs[700001:900000], maxshape=(spliced_outputs.shape[0], max_length, 4), chunks=True)
print('saved training inputs')
hf.create_dataset('y_valid', data=spliced_outputs[1700001:1900000], maxshape=(spliced_outputs.shape[0],), chunks=True)
print('saved training outputs')
hf.create_dataset('x_test', data=one_hot_inputs[900001:1000000], maxshape=(spliced_outputs.shape[0], max_length, 4), chunks=True)
print('saved training inputs')
hf.create_dataset('y_test', data=spliced_outputs[1900001:2000000], maxshape=(spliced_outputs.shape[0],), chunks=True)
print('saved training outputs')

In [ ]:
!mv dataset.h5 dataset1.h5

In [ ]:
with h5py.File("dataset.h5","r") as f:
    print(f.keys())

# 3 mil to 5.5 mil

In [ ]:
from IPython import get_ipython
get_ipython().magic('reset -sf')

In [ ]:
start = 3000000
stop = 5500000

In [ ]:
import numpy as np
import pandas as pd
import h5py
import os

In [ ]:
with open('train_sequences.txt') as handle:
    raw_data = pd.read_csv(handle, sep='\n').to_numpy()

In [ ]:
max_length = 0

def split_inputs_outputs(a):
    global max_length
    element = a.split('\t')
    seq, label = element[0], element[1]
    pre = 'TGCATTTTTTTCACATC'
    post = 'GGTTACGGCTGTT'
    seq = seq[len(pre):-len(post)]
    max_length = len(seq) if len(seq) > max_length else max_length
    return seq, label

func = np.vectorize(split_inputs_outputs)
temp = np.squeeze(raw_data)
inputs, outputs = func(temp)
outputs = outputs.astype(np.dtype('f8'))

In [ ]:
def find_Ns(s):
    if 'N' in s:
        return 0
    else:
        return 1

func = np.vectorize(find_Ns)
mask = func(inputs).astype(bool)
spliced_inputs = inputs[mask]
spliced_outputs = outputs[mask]
print(spliced_inputs.shape)

In [ ]:
def one_hot_encode(seq, L, i):
    if i % 100000 == 0:
        print(i//100000)
    seq = np.array(list(seq))
    encoded = np.zeros((L, 4))
    As = np.where(seq == 'A')[0]
    Cs = np.where(seq == 'C')[0]
    Ts = np.where(seq == 'T')[0]
    Gs = np.where(seq == 'G')[0]
    encoded[As] = np.array([1, 0, 0, 0])
    encoded[Cs] = np.array([0, 1, 0, 0])
    encoded[Gs] = np.array([0, 0, 1, 0])
    encoded[Ts] = np.array([0, 0, 0, 1])
    return encoded

order = np.arange(spliced_inputs.shape[0])
one_hot_inputs = np.vectorize(one_hot_encode, otypes=[np.ndarray])(spliced_inputs[start:stop], max_length, order[start:stop])
one_hot_inputs = np.array(one_hot_inputs.tolist(), dtype=np.dtype('f8'))
print(one_hot_inputs.shape)

In [ ]:
with h5py.File('dataset.h5', 'a') as hf:
    hf["x_train"].resize((hf["x_train"].shape[0] + one_hot_inputs.shape[0]), axis = 0)
    hf["x_train"][-one_hot_inputs.shape[0]:] = one_hot_inputs
    
    hf["y_train"].resize((hf["y_train"].shape[0] + one_hot_inputs.shape[0]), axis = 0)
    hf["y_train"][-one_hot_inputs.shape[0]:] = spliced_outputs[start:stop]

# 5.5 mil to 6.7 mil

In [ ]:
from IPython import get_ipython
get_ipython().magic('reset -sf')

In [ ]:
start = 5500000
stop = 6715197

In [ ]:
import numpy as np
import pandas as pd
import h5py
import os

In [ ]:
with open('train_sequences.txt') as handle:
    raw_data = pd.read_csv(handle, sep='\n').to_numpy()

In [ ]:
max_length = 0

def split_inputs_outputs(a):
    global max_length
    element = a.split('\t')
    seq, label = element[0], element[1]
    pre = 'TGCATTTTTTTCACATC'
    post = 'GGTTACGGCTGTT'
    seq = seq[len(pre):-len(post)]
    max_length = len(seq) if len(seq) > max_length else max_length
    return seq, label

func = np.vectorize(split_inputs_outputs)
temp = np.squeeze(raw_data)
inputs, outputs = func(temp)
outputs = outputs.astype(np.dtype('f8'))

In [ ]:
def find_Ns(s):
    if 'N' in s:
        return 0
    else:
        return 1

func = np.vectorize(find_Ns)
mask = func(inputs).astype(bool)
spliced_inputs = inputs[mask]
spliced_outputs = outputs[mask]
print(spliced_inputs.shape)

In [ ]:
def one_hot_encode(seq, L, i):
    if i % 100000 == 0:
        print(i//100000)
    seq = np.array(list(seq))
    encoded = np.zeros((L, 4))
    As = np.where(seq == 'A')[0]
    Cs = np.where(seq == 'C')[0]
    Ts = np.where(seq == 'T')[0]
    Gs = np.where(seq == 'G')[0]
    encoded[As] = np.array([1, 0, 0, 0])
    encoded[Cs] = np.array([0, 1, 0, 0])
    encoded[Gs] = np.array([0, 0, 1, 0])
    encoded[Ts] = np.array([0, 0, 0, 1])
    return encoded

order = np.arange(spliced_inputs.shape[0])
one_hot_inputs = np.vectorize(one_hot_encode, otypes=[np.ndarray])(spliced_inputs[start:stop], max_length, order[start:stop])
one_hot_inputs = np.array(one_hot_inputs.tolist(), dtype=np.dtype('f8'))
print(one_hot_inputs.shape)

In [ ]:
with h5py.File('dataset.h5', 'a') as hf:
    hf["x_train"].resize((hf["x_train"].shape[0] + one_hot_inputs.shape[0]), axis = 0)
    hf["x_train"][-one_hot_inputs.shape[0]:] = one_hot_inputs
    
    hf["y_train"].resize((hf["y_train"].shape[0] + one_hot_inputs.shape[0]), axis = 0)
    hf["y_train"][-one_hot_inputs.shape[0]:] = spliced_outputs[start:stop]

In [ ]:
with h5py.File('dataset.h5', 'a') as hf:
    print(hf['x_train'].shape)
    print(hf['y_train'].shape)